PERTANYAAN

1) Harus stemming dan lemmatization ? atau salah satu cukup ? karena lemma lebih bagus dari stemming ? urutannya harus spt apa ?
pakai salah 1 aja cukup, karena cari akar kata

2) Kalau classification yang dipelajari di junior data scientist, itu classification berdasarkan kolom2 angka, dan y = angka. sekarang x = text, dan y=angka. bagaimana caranya ?



In [1]:
# Import Library yang akan dibutuhkan dalam teks analitik
import pandas as pd
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
from wordcloud import WordCloud
from PIL import Image
import seaborn as sns
from nltk.util import ngrams
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Letakkan posisi notebook dalam folder yang smaa dengan data yang akan digunakan
%cd "/content/gdrive/MyDrive/NARASIO/SENIOR DATA SCIENTIST/MEET 5 CAPSTONE/Text Analytics/"

# Mengkonfirmasi daftar file dalam folder yang telah dipilih
%ls

/content/gdrive/MyDrive/NARASIO/SENIOR DATA SCIENTIST/MEET 5 CAPSTONE/Text Analytics
'DHANANJAYA CAPSTONE.ipynb'   pima-indians-diabetes.data.txt
 file.csv                    'TFIDFTransformer vs. TFIDFVectorizer.ipynb'
 foo.csv                      weatherAUS.csv
'Kindle Book Review.csv'      xgboost.ipynb
 metadata.txt


In [4]:
# Download corpus nltk yang akan digunakan
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
# Input Data review film dalam bahasa inggris
book_review = pd.read_csv("Kindle Book Review.csv", encoding="utf-8")
book_review.head()

,Unnamed: 0,Unnamed: 0.1,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [6]:
book_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      12000 non-null  int64 
 1   Unnamed: 0.1    12000 non-null  int64 
 2   asin            12000 non-null  object
 3   helpful         12000 non-null  object
 4   rating          12000 non-null  int64 
 5   reviewText      12000 non-null  object
 6   reviewTime      12000 non-null  object
 7   reviewerID      12000 non-null  object
 8   reviewerName    11962 non-null  object
 9   summary         12000 non-null  object
 10  unixReviewTime  12000 non-null  int64 
dtypes: int64(4), object(7)
memory usage: 1.0+ MB


In [7]:
def ubah_rating(rating) :
    if rating > 3 :
        return 2
    if rating == 3:
        return 1
    else :
        return 0

In [8]:
book_review['rating_prep'] = book_review.rating.apply(lambda x : ubah_rating(int(x)))

In [9]:
book_review.head()

,Unnamed: 0,Unnamed: 0.1,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,rating_prep
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600,1
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400,2
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400,1
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400,1
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000,2


In [10]:
data_select = book_review[['reviewerName', 'summary', 'reviewText', 'rating','rating_prep']]
data_select.head()

,reviewerName,summary,reviewText,rating,rating_prep
0,Ridley,Entertaining But Average,"Jace Rankin may be short, but he's nothing to ...",3,1
1,Holly Butler,Terrific menage scenes!,Great short read. I didn't want to put it dow...,5,2
2,Merissa,Snapdragon Alley,I'll start by saying this is the first of four...,3,1
3,Cleargrace,very light murder cozy,Aggie is Angela Lansbury who carries pocketboo...,3,1
4,Rjostler,Book,I did not expect this type of book to be in li...,4,2


In [11]:
data_select.rating_prep.value_counts()

2    6000
0    4000
1    2000
Name: rating_prep, dtype: int64

In [12]:
data_select.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   reviewerName  11962 non-null  object
 1   summary       12000 non-null  object
 2   reviewText    12000 non-null  object
 3   rating        12000 non-null  int64 
 4   rating_prep   12000 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 468.9+ KB


LOWERCASE

In [13]:
# merubah input teks menjadi huruf kecil semua
data_select["reviewText_low"] = data_select["reviewText"].apply(lambda x: x.lower())
data_select.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,reviewerName,summary,reviewText,rating,rating_prep,reviewText_low
0,Ridley,Entertaining But Average,"Jace Rankin may be short, but he's nothing to ...",3,1,"jace rankin may be short, but he's nothing to ..."
1,Holly Butler,Terrific menage scenes!,Great short read. I didn't want to put it dow...,5,2,great short read. i didn't want to put it dow...
2,Merissa,Snapdragon Alley,I'll start by saying this is the first of four...,3,1,i'll start by saying this is the first of four...
3,Cleargrace,very light murder cozy,Aggie is Angela Lansbury who carries pocketboo...,3,1,aggie is angela lansbury who carries pocketboo...
4,Rjostler,Book,I did not expect this type of book to be in li...,4,2,i did not expect this type of book to be in li...


REMOVE PUNCTUATION

In [14]:
punct = re.compile(r"[^\w\s]")
# Menghilangkan angka
number = re.compile(r"\d")
# Menghilangkan double spasi
#space = re.compile(r"\s\s+")

def clean_data(text):
    text = re.sub(punct, " ", text)
    text = re.sub(number, " ", text)
    #text = re.sub(space, "", text)
    return text

In [15]:
# Memanggil function data cleaning untuk kemudian diterapkan dalam dataframe
data_select["reviewText_removepunc"] = data_select["reviewText_low"].apply(lambda x: clean_data(x))
data_select.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,reviewerName,summary,reviewText,rating,rating_prep,reviewText_low,reviewText_removepunc
0,Ridley,Entertaining But Average,"Jace Rankin may be short, but he's nothing to ...",3,1,"jace rankin may be short, but he's nothing to ...",jace rankin may be short but he s nothing to ...
1,Holly Butler,Terrific menage scenes!,Great short read. I didn't want to put it dow...,5,2,great short read. i didn't want to put it dow...,great short read i didn t want to put it dow...
2,Merissa,Snapdragon Alley,I'll start by saying this is the first of four...,3,1,i'll start by saying this is the first of four...,i ll start by saying this is the first of four...
3,Cleargrace,very light murder cozy,Aggie is Angela Lansbury who carries pocketboo...,3,1,aggie is angela lansbury who carries pocketboo...,aggie is angela lansbury who carries pocketboo...
4,Rjostler,Book,I did not expect this type of book to be in li...,4,2,i did not expect this type of book to be in li...,i did not expect this type of book to be in li...


CEK DUPLICATE

In [16]:
# Cek duplicate secara keseluruhan dataframe
duplicates = data_select[data_select.duplicated()]
duplicates

,reviewerName,summary,reviewText,rating,rating_prep,reviewText_low,reviewText_removepunc


REMOVE STOPWORDS

In [17]:
# Akan digunakan list stopwords dari nltk
stop_words = set(stopwords.words('english'))

In [18]:
# Token summary
data_select["cleanReviewText"] = data_select["reviewText_removepunc"].apply(lambda x: word_tokenize(x))

data_select.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,reviewerName,summary,reviewText,rating,rating_prep,reviewText_low,reviewText_removepunc,cleanReviewText
0,Ridley,Entertaining But Average,"Jace Rankin may be short, but he's nothing to ...",3,1,"jace rankin may be short, but he's nothing to ...",jace rankin may be short but he s nothing to ...,"[jace, rankin, may, be, short, but, he, s, not..."
1,Holly Butler,Terrific menage scenes!,Great short read. I didn't want to put it dow...,5,2,great short read. i didn't want to put it dow...,great short read i didn t want to put it dow...,"[great, short, read, i, didn, t, want, to, put..."
2,Merissa,Snapdragon Alley,I'll start by saying this is the first of four...,3,1,i'll start by saying this is the first of four...,i ll start by saying this is the first of four...,"[i, ll, start, by, saying, this, is, the, firs..."
3,Cleargrace,very light murder cozy,Aggie is Angela Lansbury who carries pocketboo...,3,1,aggie is angela lansbury who carries pocketboo...,aggie is angela lansbury who carries pocketboo...,"[aggie, is, angela, lansbury, who, carries, po..."
4,Rjostler,Book,I did not expect this type of book to be in li...,4,2,i did not expect this type of book to be in li...,i did not expect this type of book to be in li...,"[i, did, not, expect, this, type, of, book, to..."


In [19]:
# Menerapkan list stopwords (stopwords berisi stopwords dari nltk dan tambahan dari kondisi data)
data_select["cleanReviewText2"] = data_select["cleanReviewText"].apply(lambda x: [word for word in x if word not in stop_words])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
data_select.head()

,reviewerName,summary,reviewText,rating,rating_prep,reviewText_low,reviewText_removepunc,cleanReviewText,cleanReviewText2
0,Ridley,Entertaining But Average,"Jace Rankin may be short, but he's nothing to ...",3,1,"jace rankin may be short, but he's nothing to ...",jace rankin may be short but he s nothing to ...,"[jace, rankin, may, be, short, but, he, s, not...","[jace, rankin, may, short, nothing, mess, man,..."
1,Holly Butler,Terrific menage scenes!,Great short read. I didn't want to put it dow...,5,2,great short read. i didn't want to put it dow...,great short read i didn t want to put it dow...,"[great, short, read, i, didn, t, want, to, put...","[great, short, read, want, put, read, one, sit..."
2,Merissa,Snapdragon Alley,I'll start by saying this is the first of four...,3,1,i'll start by saying this is the first of four...,i ll start by saying this is the first of four...,"[i, ll, start, by, saying, this, is, the, firs...","[start, saying, first, four, books, expecting,..."
3,Cleargrace,very light murder cozy,Aggie is Angela Lansbury who carries pocketboo...,3,1,aggie is angela lansbury who carries pocketboo...,aggie is angela lansbury who carries pocketboo...,"[aggie, is, angela, lansbury, who, carries, po...","[aggie, angela, lansbury, carries, pocketbooks..."
4,Rjostler,Book,I did not expect this type of book to be in li...,4,2,i did not expect this type of book to be in li...,i did not expect this type of book to be in li...,"[i, did, not, expect, this, type, of, book, to...","[expect, type, book, library, pleased, find, p..."


In [ ]:
data_select.cleanReviewText.head(10)

0    [jace, rankin, may, be, short, but, he, s, not...
1    [great, short, read, i, didn, t, want, to, put...
2    [i, ll, start, by, saying, this, is, the, firs...
3    [aggie, is, angela, lansbury, who, carries, po...
4    [i, did, not, expect, this, type, of, book, to...
5    [aislinn, is, a, little, girl, with, big, drea...
6    [this, has, the, makings, of, a, good, story, ...
7    [i, got, this, because, i, like, collaborated,...
8    [loved, this, book, i, am, hooked, on, this, s...
9    [and, that, s, a, good, thing, short, sweet, t...
Name: cleanReviewText, dtype: object

In [ ]:
data_select.cleanReviewText2.head(10)

0    [jace, rankin, may, short, nothing, mess, man,...
1    [great, short, read, want, put, read, one, sit...
2    [start, saying, first, four, books, expecting,...
3    [aggie, angela, lansbury, carries, pocketbooks...
4    [expect, type, book, library, pleased, find, p...
5    [aislinn, little, girl, big, dreams, death, ol...
6    [makings, good, story, unfortunately, disappoi...
7    [got, like, collaborated, short, stories, alot...
8    [loved, book, hooked, series, hope, kelsey, ma...
9    [good, thing, short, sweet, tease, gives, ever...
Name: cleanReviewText2, dtype: object

LEMMATIZATION AND STEMMING

In [20]:
# Lemmatization
lemmatizer = WordNetLemmatizer()
data_select["lemmatizer_review"] = data_select["cleanReviewText2"].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# display data
data_select.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,reviewerName,summary,reviewText,rating,rating_prep,reviewText_low,reviewText_removepunc,cleanReviewText,cleanReviewText2,lemmatizer_review
0,Ridley,Entertaining But Average,"Jace Rankin may be short, but he's nothing to ...",3,1,"jace rankin may be short, but he's nothing to ...",jace rankin may be short but he s nothing to ...,"[jace, rankin, may, be, short, but, he, s, not...","[jace, rankin, may, short, nothing, mess, man,...","[jace, rankin, may, short, nothing, mess, man,..."
1,Holly Butler,Terrific menage scenes!,Great short read. I didn't want to put it dow...,5,2,great short read. i didn't want to put it dow...,great short read i didn t want to put it dow...,"[great, short, read, i, didn, t, want, to, put...","[great, short, read, want, put, read, one, sit...","[great, short, read, want, put, read, one, sit..."
2,Merissa,Snapdragon Alley,I'll start by saying this is the first of four...,3,1,i'll start by saying this is the first of four...,i ll start by saying this is the first of four...,"[i, ll, start, by, saying, this, is, the, firs...","[start, saying, first, four, books, expecting,...","[start, saying, first, four, book, expecting, ..."
3,Cleargrace,very light murder cozy,Aggie is Angela Lansbury who carries pocketboo...,3,1,aggie is angela lansbury who carries pocketboo...,aggie is angela lansbury who carries pocketboo...,"[aggie, is, angela, lansbury, who, carries, po...","[aggie, angela, lansbury, carries, pocketbooks...","[aggie, angela, lansbury, carry, pocketbook, i..."
4,Rjostler,Book,I did not expect this type of book to be in li...,4,2,i did not expect this type of book to be in li...,i did not expect this type of book to be in li...,"[i, did, not, expect, this, type, of, book, to...","[expect, type, book, library, pleased, find, p...","[expect, type, book, library, pleased, find, p..."


In [21]:

# Stemming
ps = PorterStemmer()
data_select["stem_review"] = data_select["lemmatizer_review"].apply(lambda x: [ps.stem(word) for word in x])

# display data
data_select.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,reviewerName,summary,reviewText,rating,rating_prep,reviewText_low,reviewText_removepunc,cleanReviewText,cleanReviewText2,lemmatizer_review,stem_review
0,Ridley,Entertaining But Average,"Jace Rankin may be short, but he's nothing to ...",3,1,"jace rankin may be short, but he's nothing to ...",jace rankin may be short but he s nothing to ...,"[jace, rankin, may, be, short, but, he, s, not...","[jace, rankin, may, short, nothing, mess, man,...","[jace, rankin, may, short, nothing, mess, man,...","[jace, rankin, may, short, noth, mess, man, ha..."
1,Holly Butler,Terrific menage scenes!,Great short read. I didn't want to put it dow...,5,2,great short read. i didn't want to put it dow...,great short read i didn t want to put it dow...,"[great, short, read, i, didn, t, want, to, put...","[great, short, read, want, put, read, one, sit...","[great, short, read, want, put, read, one, sit...","[great, short, read, want, put, read, one, sit..."
2,Merissa,Snapdragon Alley,I'll start by saying this is the first of four...,3,1,i'll start by saying this is the first of four...,i ll start by saying this is the first of four...,"[i, ll, start, by, saying, this, is, the, firs...","[start, saying, first, four, books, expecting,...","[start, saying, first, four, book, expecting, ...","[start, say, first, four, book, expect, conclu..."
3,Cleargrace,very light murder cozy,Aggie is Angela Lansbury who carries pocketboo...,3,1,aggie is angela lansbury who carries pocketboo...,aggie is angela lansbury who carries pocketboo...,"[aggie, is, angela, lansbury, who, carries, po...","[aggie, angela, lansbury, carries, pocketbooks...","[aggie, angela, lansbury, carry, pocketbook, i...","[aggi, angela, lansburi, carri, pocketbook, in..."
4,Rjostler,Book,I did not expect this type of book to be in li...,4,2,i did not expect this type of book to be in li...,i did not expect this type of book to be in li...,"[i, did, not, expect, this, type, of, book, to...","[expect, type, book, library, pleased, find, p...","[expect, type, book, library, pleased, find, p...","[expect, type, book, librari, pleas, find, pri..."


In [ ]:
# Membandingkan 5 data awal sebelum dan sesudah cleaning
for i, j in enumerate(data_select['cleanReviewText2'][:5]):
  print("clean      :", data_select["cleanReviewText2"][i])
  print("lemmatizer :", data_select["lemmatizer_review"][i])
  print("stem       :", data_select["stem_review"][i])
  print("\n")
  
  

clean      : ['jace', 'rankin', 'may', 'short', 'nothing', 'mess', 'man', 'hauled', 'saloon', 'undertaker', 'knows', 'famous', 'bounty', 'hunter', 'oregon', 'shot', 'man', 'saloon', 'finished', 'years', 'long', 'quest', 'avenge', 'sister', 'murder', 'trying', 'figure', 'next', 'snotty', 'nosed', 'farm', 'boy', 'rescued', 'gang', 'bullies', 'offers', 'money', 'kill', 'man', 'forced', 'ranch', 'reluctantly', 'agrees', 'bring', 'man', 'justice', 'kill', 'outright', 'first', 'needs', 'tell', 'sister', 'widower', 'news', 'kyla', 'kyle', 'springer', 'bailey', 'riding', 'trails', 'sleeping', 'ground', 'past', 'month', 'trying', 'find', 'jace', 'wants', 'revenge', 'man', 'killed', 'husband', 'took', 'ranch', 'amongst', 'crimes', 'keen', 'detour', 'jace', 'wants', 'take', 'realizes', 'options', 'hides', 'behind', 'boy', 'persona', 'best', 'tries', 'keep', 'pace', 'confrontation', 'along', 'way', 'gets', 'shot', 'jace', 'discovers', 'kyle', 'kyla', 'come', 'clean', 'whole', 'reason', 'needs', 's

In [22]:
# Menggabungkan kembali kata yang ditoken
data_select["cleanReviewText2"] = data_select["cleanReviewText2"].apply(lambda x: " ".join(x))
data_select["stem_review_join"] = data_select["stem_review"].apply(lambda x: " ".join(x))
data_select.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,reviewerName,summary,reviewText,rating,rating_prep,reviewText_low,reviewText_removepunc,cleanReviewText,cleanReviewText2,lemmatizer_review,stem_review,stem_review_join
0,Ridley,Entertaining But Average,"Jace Rankin may be short, but he's nothing to ...",3,1,"jace rankin may be short, but he's nothing to ...",jace rankin may be short but he s nothing to ...,"[jace, rankin, may, be, short, but, he, s, not...",jace rankin may short nothing mess man hauled ...,"[jace, rankin, may, short, nothing, mess, man,...","[jace, rankin, may, short, noth, mess, man, ha...",jace rankin may short noth mess man haul saloo...
1,Holly Butler,Terrific menage scenes!,Great short read. I didn't want to put it dow...,5,2,great short read. i didn't want to put it dow...,great short read i didn t want to put it dow...,"[great, short, read, i, didn, t, want, to, put...",great short read want put read one sitting sex...,"[great, short, read, want, put, read, one, sit...","[great, short, read, want, put, read, one, sit...",great short read want put read one sit sex sce...
2,Merissa,Snapdragon Alley,I'll start by saying this is the first of four...,3,1,i'll start by saying this is the first of four...,i ll start by saying this is the first of four...,"[i, ll, start, by, saying, this, is, the, firs...",start saying first four books expecting conclu...,"[start, saying, first, four, book, expecting, ...","[start, say, first, four, book, expect, conclu...",start say first four book expect conclud cente...
3,Cleargrace,very light murder cozy,Aggie is Angela Lansbury who carries pocketboo...,3,1,aggie is angela lansbury who carries pocketboo...,aggie is angela lansbury who carries pocketboo...,"[aggie, is, angela, lansbury, who, carries, po...",aggie angela lansbury carries pocketbooks inst...,"[aggie, angela, lansbury, carry, pocketbook, i...","[aggi, angela, lansburi, carri, pocketbook, in...",aggi angela lansburi carri pocketbook instead ...
4,Rjostler,Book,I did not expect this type of book to be in li...,4,2,i did not expect this type of book to be in li...,i did not expect this type of book to be in li...,"[i, did, not, expect, this, type, of, book, to...",expect type book library pleased find price right,"[expect, type, book, library, pleased, find, p...","[expect, type, book, librari, pleas, find, pri...",expect type book librari pleas find price right


## MODELLING



1) Split dulu baru tfidf, atau tfidf dulu baru split ?

2) XTrain itu pakai kata-kata yang tokenized atau yang sudah digabung


TRAIN TEST SPLIT & TFIDF

In [23]:
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, KFold, GridSearchCV

In [90]:
X_train, X_test, y_train, y_test = train_test_split(data_select["stem_review_join"], data_select["rating_prep"], test_size=0.2, random_state=24)

In [91]:
X_train.shape

(9600,)

In [92]:
X_test.shape

(2400,)

In [93]:
# Mencari tf-idf dengan menggunakan unigram untuk X_train
tfidf_vect_1 = TfidfVectorizer(ngram_range=(1,1))
X_train_tfidf = tfidf_vect_1.fit_transform(X_train.values.tolist())
X_train_tfidf.shape

(9600, 17988)

In [ ]:
#melihat bentuk matrix
#pd.DataFrame(X_train_tfidf).to_csv("file.csv")

*Kemarin diminta agar xtest nya juga di tfidf dengan fit transform, jadinya jumlah feature xtrain dan xtest berbeda, dan ga bisa dijalankan di model.predict.*

In [94]:
# Mencari tf-idf dengan menggunakan unigram untuk X_test
'''
tfidf_vect_2 = TfidfVectorizer(ngram_range=(1,1))
X_test_tfidf = tfidf_vect_2.fit_transform(X_test.values.tolist())
X_test_tfidf.shape
'''

(2400, 9569)

Setelah googling, dapat penjelasan ini : https://stackoverflow.com/questions/40731271/test-and-train-dataset-has-different-number-of-features

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer(binary=True, use_idf=True)
X_train_tfidf = vec.fit_transform(X_train) 
X_test_tfidf = vec.transform(X_test)


1. MULTINOMIAL NAIVE BAYES

In [122]:
#modelling svm, random forest, naive bayes
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()

In [123]:
model.fit(X_train_tfidf, y_train)


MultinomialNB()

In [124]:
y_predict = model.predict(X_test_tfidf)

In [125]:
#kalau displit dulu baru tfidf, acuracy = 0.65
accuracy = accuracy_score(y_test, y_predict)
accuracy

0.6508333333333334

In [130]:
 from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_predict, labels=[0, 1, 2]))

              precision    recall  f1-score   support

           0       0.84      0.48      0.61       834
           1       0.00      0.00      0.00       387
           2       0.60      0.99      0.75      1179

    accuracy                           0.65      2400
   macro avg       0.48      0.49      0.45      2400
weighted avg       0.59      0.65      0.58      2400



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [131]:
print(confusion_matrix(y_test, y_predict))

[[ 398    0  436]
 [  59    0  328]
 [  15    0 1164]]


tuning hyperparameter, bandingkan dengan beberapa model multinominal naive bayes, random forest, xgboost. bandingkan juga dengan pengaruh tfidf unigram bigram, trigram, tampilkan classification report

2. XGBOOST

In [102]:
import xgboost as xgb
from xgboost import XGBClassifier


In [103]:
# fit model no training data
model2 = XGBClassifier()
model2.fit(X_train_tfidf, y_train)

XGBClassifier(objective='multi:softprob')

In [104]:
y_predict = model2.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_predict)
accuracy

0.6758333333333333

In [120]:

print(classification_report(y_test, y_predict, labels=[0, 1, 2]))

              precision    recall  f1-score   support

           0       0.76      0.68      0.72       834
           1       0.67      0.01      0.02       387
           2       0.68      0.95      0.79      1179

    accuracy                           0.70      2400
   macro avg       0.70      0.55      0.51      2400
weighted avg       0.70      0.70      0.64      2400



3. RANDOM FOREST

In [116]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
model3=RandomForestClassifier(n_estimators=500)

#Train the model using the training sets y_pred=clf.predict(X_test)
model3.fit(X_train_tfidf,y_train)

y_predict=model3.predict(X_test_tfidf)

In [117]:
accuracy = accuracy_score(y_test, y_predict)
accuracy

0.7029166666666666

In [121]:

print(classification_report(y_test, y_predict, labels=[0, 1, 2]))

              precision    recall  f1-score   support

           0       0.76      0.68      0.72       834
           1       0.67      0.01      0.02       387
           2       0.68      0.95      0.79      1179

    accuracy                           0.70      2400
   macro avg       0.70      0.55      0.51      2400
weighted avg       0.70      0.70      0.64      2400

